# Dokumentation des Datenbereinigungsprozesses

## 1. Einführung

Dieses Dokument beschreibt den Prozess der Datenbereinigung und -vorverarbeitung, der auf den Hautbilddatensatz angewendet wird. Die Datenbereinigung ist ein grundlegender Schritt in der Datenanalyse und der Erstellung von Machine-Learning-Modellen, da sie die Qualität, Konsistenz und Repräsentativität der verwendeten Informationen sicherstellt.

## 2. Projekteinrichtung

Der verwendete Datensatz liegt im CSV-Format vor und ist unter folgendem Pfad gespeichert:

```
data/ISIC-images/metadata.csv
```

Um die Überschreibung vorheriger Dateien zu vermeiden, wird ein dynamischer Zeitstempel für die Ausgabedateien generiert:

- **Bereinigter Datensatz:** `metadata_clean_<TIMESTAMP>.csv`
- **Datensatz mit unbekannten Labels:** `metadata_unknown_<TIMESTAMP>.csv`
- **Protokolldatei:** `cleaning_log_<TIMESTAMP>.log`
- **Label-Mapping:** `label_mapping.json`

Zusätzlich werden Konfigurationsparameter definiert, die die Vorverarbeitung anpassbar machen:

- Entfernung von Duplikaten.
- Normalisierung des Alters.
- Kodierung kategorischer Variablen.

## 3. Datenbereinigungsprozess

### 3.1 Erkennung und Behandlung von Ausreißern

Die Methode **Interquartile Range (IQR)** wird verwendet, um Ausreißer in numerischen Variablen zu erkennen und zu ersetzen. Wenn Werte außerhalb des durch den IQR definierten Bereichs erkannt werden, werden diese durch den Median der Verteilung ersetzt, um extreme Verzerrungen zu vermeiden.

### 3.2 Entfernung von Duplikaten

Duplikate werden identifiziert und entfernt, um Redundanzen in der Analyse zu vermeiden. Die Entfernung erfolgt unter der Sicherstellung, dass keine kritischen Daten verloren gehen.

### 3.3 Behandlung fehlender Werte

Es werden spezifische Strategien für jeden Variablentyp angewendet:

- **`age_approx`**: Wird mit dem Median imputiert und normalisiert, falls die Option aktiviert ist.
- **`sex`**: Wird mit dem Modus (häufigster Wert) gefüllt und in eine binäre Variable umgewandelt (0 für männlich und 1 für weiblich).
- **`anatom_site_general`**: Fehlende Werte werden durch "unknown" ersetzt und kodiert, falls die Option aktiviert ist.
- **`benign_malignant`**: Fehlende Werte werden als "unknown" markiert, jedoch wird keine Inferenz durchgeführt, um Fehler in der Klassifikation zu vermeiden.

### 3.4 Normalisierung numerischer Variablen

Falls die Normalisierungsoption aktiviert ist, wird die Variable `age_approx` mit der **Min-Max-Scaling**-Technik transformiert, die die Werte in den Bereich `[0,1]` skaliert, um die Konvergenz in Machine-Learning-Modellen zu verbessern.

### 3.5 Kodierung kategorischer Variablen

Um die Verwendung in numerischen Modellen zu erleichtern, werden kategorische Variablen in numerische Darstellungen umgewandelt:

- **`sex`**: Wird als 0 für männlich und 1 für weiblich kodiert.
- **`anatom_site_general`**: Wird in numerische Codes basierend auf eindeutigen Kategorien umgewandelt.
- **`diagnosis`**: Es wird ein Label-Mapping erstellt und in `label_mapping.json` gespeichert.

### 3.6 Entfernung irrelevanter Spalten

Spalten, die keinen Wert für die Analyse oder das Vorhersagemodell bieten, werden entfernt, wie z.B.:

```
attribution, copyright_license, anatom_site_special, concomitant_biopsy,
diagnosis_1, diagnosis_2, diagnosis_3, diagnosis_confirm_type, image_type, lesion_id
```

### 3.7 Aufteilung des Datensatzes

Der Datensatz wird in zwei Teilmengen aufgeteilt:

- **`metadata_clean_*.csv`**: Enthält Datensätze, bei denen `benign_malignant` einen bekannten Wert hat (geeignet für das Training).
- **`metadata_unknown_*.csv`**: Enthält Datensätze, bei denen `benign_malignant` als "unknown" markiert ist (kann für explorative Analysen oder Clustering verwendet werden).

## 4. Generierung von Protokollen

Der Bereinigungsprozess generiert eine Protokolldatei (`cleaning_log_<TIMESTAMP>.log`), die alle durchgeführten Aktionen dokumentiert, einschließlich:

- Anzahl der entfernten Duplikate.
- Anzahl der erkannten und korrigierten Ausreißer.
- Anzahl der Datensätze in den finalen Datensätzen.
- Mögliche Fehler, die während des Prozesses aufgetreten sind.

## 5. Bewertung der Datenqualität

Es werden Funktionen bereitgestellt, um die Verteilung der Klassen vor und nach der Bereinigung zu visualisieren. Dies ermöglicht die Bewertung der Effektivität der Vorverarbeitung und stellt sicher, dass die Labels ihre Repräsentativität behalten.

## 6. Ethische und wissenschaftliche Überlegungen

Die Vorverarbeitung medizinischer Daten muss unter strengen ethischen Gesichtspunkten erfolgen:

- Es wird keine Imputation für `benign_malignant` durchgeführt, um Verzerrungen in der Klassifikation zu vermeiden.
- Alle Änderungen werden dokumentiert, um die Nachvollziehbarkeit und Reproduzierbarkeit der Analyse zu gewährleisten.
- Datensätze werden nicht aufgrund von Alters-, Geschlechts- oder anatomischen Verzerrungen entfernt, um die Diversität des Datensatzes zu bewahren.
- Irrelevante Daten werden entfernt, um Overfitting zu vermeiden und generalisierbare Modelle zu gewährleisten.

## 7. Fazit

Dieser Bereinigungs- und Vorverarbeitungsprozess stellt sicher, dass die Daten von hoher Qualität sind und für die Verwendung in Machine-Learning-Modellen bereit sind. Durch die Anwendung von Normalisierungs-, Kodierungs- und Nullwertbehandlungstechniken wird die Repräsentativität der Merkmale optimiert und Verzerrungen in der prädiktiven Analyse minimiert. Die Generierung von Protokollen (Logs) gewährleistet die Nachvollziehbarkeit und Reproduzierbarkeit des Prozesses und erfüllt ethische und wissenschaftliche Standards im Umgang mit medizinischen Daten.